Данный ноутбук является частью проекта "Рекомендательные системы с элементами NLP" команды МОВС ВШЭ

# Tries to see metrics

In [24]:
from datetime import datetime
from matplotlib import pyplot as plt
from scipy import stats

import seaborn as sb
import requests

from pprint import pprint

import numpy as np
import pandas as pd

from implicit.nearest_neighbours import TFIDFRecommender, BM25Recommender
from implicit.als import AlternatingLeastSquares

from rectools import Columns
from rectools.dataset import Dataset
from rectools.metrics import Precision, Recall, MeanInvUserFreq, Serendipity, calc_metrics, MAP
from rectools.models import ImplicitItemKNNWrapperModel, RandomModel, PopularModel
from rectools.model_selection import TimeRangeSplitter, cross_validate

## Загружаем данные

In [3]:
# Загрузим в ноутбук
!gdown 1I_uFqSR7TiGp7BfKH6wwnpmiBk1ZAzww
!gdown 1CP4deOmpaOqnwdnqZCcZBqpZjBxKDs6l
!gdown 1S0Wz20_d-HV9USfxwqmE4FRtkOovs2XK

Downloading...
From: https://drive.google.com/uc?id=1I_uFqSR7TiGp7BfKH6wwnpmiBk1ZAzww
To: /content/items.csv
100% 31.8M/31.8M [00:01<00:00, 29.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1CP4deOmpaOqnwdnqZCcZBqpZjBxKDs6l
To: /content/users.csv
100% 29.9M/29.9M [00:00<00:00, 37.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1S0Wz20_d-HV9USfxwqmE4FRtkOovs2XK
To: /content/interactions.csv
100% 180M/180M [00:04<00:00, 36.5MB/s]


In [4]:
# Просмотры контекта пользователями
items_df = pd.read_csv('/content/items.csv')
items_df.head()

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю..."
4,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж..."


In [5]:
# Пользователи
users_df = pd.read_csv('/content/users.csv')
users_df.head()

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
2,1047345,age_45_54,income_40_60,Ж,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0


In [6]:
# Фильмы
interactions_df = pd.read_csv('/content/interactions.csv')
interactions_df.head()

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0


##    Датафрейм про юзеров + фильмы

In [7]:
rec_df = (
    interactions_df[['user_id', 'item_id', 'watched_pct', 'last_watch_dt']]
    .join(
        items_df[['item_id', 'release_year', 'for_kids', 'age_rating', 'genres', 'countries', 'keywords', 'title', 'description']],
        on=['item_id'],
        how='inner',
        rsuffix='inter'
    )
)

rec_df = rec_df.drop('item_idinter', axis =1)
#
# .join(
#     users_df[['user_id', 'age', 'income', 'sex', 'kids_flg']],
#     on=['item_id'],
#     how='inner',
#     rsuffix='item'
# )

In [8]:
rec_df['for_kids'] = rec_df['for_kids'].fillna(0)

In [9]:
# users_explor = users_explor.dropna()

In [10]:
# users_explor['age'] = users_explor['age'].apply(lambda x: (int(x[4:6]) + int(x[7:9])) / 2 if x != 'age_65_inf' else 65)

In [11]:
# def inc(x):
#     if x == 'income_0_20':
#         return 10
#     elif x == 'income_150_inf':
#         return 200
#     elif x == 'income_90_150':
#         return 120
#     else:
#         return (int(x[7:9]) + int(x[10:12])) / 2

In [12]:
# users_explor['income'] = users_explor['income'].apply(inc)

In [13]:
def genre(x):
    lst = x.split(',')
    map(str.strip(), x.split(','))
    return lst

In [14]:
rec_df['genres'] = rec_df['genres'].apply(lambda x: list(map(str.strip, x.split(','))))

## Item to item Finding Movie Similarities

### Try model KNN

In [15]:
rec_df.head(5)

,user_id,item_id,watched_pct,last_watch_dt,release_year,for_kids,age_rating,genres,countries,keywords,title,description
0,176549,9506,72.0,2021-05-11,2010.0,0.0,16.0,"[боевики, зарубежные, фэнтези, приключения]",Перу,"Эльдорадо, часть, 1, поисках, Храма, Солнца, 2...",Эльдорадо: часть 1. В поисках Храма Солнца,"Шэйн Уэст («Спеши любить»), Мисси Пайл («Крупн..."
4,964868,9506,100.0,2021-04-30,2010.0,0.0,16.0,"[боевики, зарубежные, фэнтези, приключения]",Перу,"Эльдорадо, часть, 1, поисках, Храма, Солнца, 2...",Эльдорадо: часть 1. В поисках Храма Солнца,"Шэйн Уэст («Спеши любить»), Мисси Пайл («Крупн..."
445,450170,9506,14.0,2021-07-18,2010.0,0.0,16.0,"[боевики, зарубежные, фэнтези, приключения]",Перу,"Эльдорадо, часть, 1, поисках, Храма, Солнца, 2...",Эльдорадо: часть 1. В поисках Храма Солнца,"Шэйн Уэст («Спеши любить»), Мисси Пайл («Крупн..."
1741,683820,9506,44.0,2021-06-28,2010.0,0.0,16.0,"[боевики, зарубежные, фэнтези, приключения]",Перу,"Эльдорадо, часть, 1, поисках, Храма, Солнца, 2...",Эльдорадо: часть 1. В поисках Храма Солнца,"Шэйн Уэст («Спеши любить»), Мисси Пайл («Крупн..."
3681,547714,9506,93.0,2021-08-07,2010.0,0.0,16.0,"[боевики, зарубежные, фэнтези, приключения]",Перу,"Эльдорадо, часть, 1, поисках, Храма, Солнца, 2...",Эльдорадо: часть 1. В поисках Храма Солнца,"Шэйн Уэст («Спеши любить»), Мисси Пайл («Крупн..."


In [16]:
rec_trunc_df = rec_df[['item_id', 'user_id', 'watched_pct', 'last_watch_dt']].rename(
    columns={'watched_pct': 'weight', 'last_watch_dt': 'datetime'}
    ).reset_index(drop=True)

In [17]:
rec_trunc_df.head()

,item_id,user_id,weight,datetime
0,9506,176549,72.0,2021-05-11
1,9506,964868,100.0,2021-04-30
2,9506,450170,14.0,2021-07-18
3,9506,683820,44.0,2021-06-28
4,9506,547714,93.0,2021-08-07


In [18]:
rec_trunc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5315821 entries, 0 to 5315820
Data columns (total 4 columns):
 #   Column    Dtype  
---  ------    -----  
 0   item_id   int64  
 1   user_id   int64  
 2   weight    float64
 3   datetime  object 
dtypes: float64(1), int64(2), object(1)
memory usage: 162.2+ MB


In [19]:
# Prepare a dataset to build a model
dataset = Dataset.construct(rec_trunc_df)

In [100]:
%%time
# Fit model and generate recommendations for all users
model = ImplicitItemKNNWrapperModel(TFIDFRecommender(K=10))
model.fit(dataset)
recos = model.recommend(
    users=rec_trunc_df[Columns.User].unique(),
    dataset=dataset,
    k=10,
    filter_viewed=True,
)

/usr/local/lib/python3.10/dist-packages/implicit/nearest_neighbours.py:233: RuntimeWarning: invalid value encountered in divide
  X.data = X.data / sqrt(bincount(X.row, X.data**2))[X.row]


CPU times: user 2min 45s, sys: 2.85 s, total: 2min 48s
Wall time: 2min 47s


In [102]:
recos

,user_id,item_id,score,rank
0,176549,11749,66.701070,1
1,176549,334,53.974913,2
2,176549,11985,46.763361,3
3,176549,13256,46.451247,4
4,176549,13586,45.474620,5
...,...,...,...,...
9313262,99387,7224,0.000000,5
9313263,99387,2800,NaN,6
9313264,99387,4466,0.000000,7
9313265,99387,8175,NaN,8


### Try cross validation with different models and metric clculations

In [23]:
min(rec_trunc_df['datetime'])

'2021-03-13'

In [22]:
max(rec_trunc_df['datetime'])

'2021-08-22'

In [27]:
splitter = TimeRangeSplitter(
    test_size="7D",
    n_splits=1,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=True,
)

In [28]:
splitter.get_test_fold_borders(dataset.interactions)

[(Timestamp('2021-08-16 00:00:00', freq='7D'),
  Timestamp('2021-08-23 00:00:00', freq='7D'))]

In [29]:
# Take few simple models to compare
models = {
    "random": RandomModel(random_state=42),
    "popular": PopularModel(),
    "most_raited": PopularModel(popularity="sum_weight"),
    "tfidf_k=5": ImplicitItemKNNWrapperModel(model=TFIDFRecommender(K=5)),
    "tfidf_k=10": ImplicitItemKNNWrapperModel(model=TFIDFRecommender(K=10)),
    "bm25_k=10_k1=0.05_b=0.1": ImplicitItemKNNWrapperModel(model=BM25Recommender(K=5, K1=0.05, B=0.1)),
}

# We will calculate several classic (precision@k and recall@k) and "beyond accuracy" metrics
metrics = {
    "prec@1": Precision(k=1),
    "prec@10": Precision(k=10),
    "map@10": MAP(k=10), #.calc_per_user(reco, interactions)
    "recall": Recall(k=10),
    "novelty": MeanInvUserFreq(k=10),
    "serendipity": Serendipity(k=10),
}

K_RECS = 10

In [30]:
%%time

# For each fold generate train and test part of dataset
# Then fit every model, generate recommendations and calculate metrics

cv_results = cross_validate(
    dataset=dataset,
    splitter=splitter,
    models=models,
    metrics=metrics,
    k=K_RECS,
    filter_viewed=True,
)

/usr/local/lib/python3.10/dist-packages/implicit/nearest_neighbours.py:233: RuntimeWarning: invalid value encountered in divide
  X.data = X.data / sqrt(bincount(X.row, X.data**2))[X.row]
/usr/local/lib/python3.10/dist-packages/implicit/nearest_neighbours.py:233: RuntimeWarning: invalid value encountered in divide
  X.data = X.data / sqrt(bincount(X.row, X.data**2))[X.row]


CPU times: user 2min 24s, sys: 3.54 s, total: 2min 27s
Wall time: 2min 26s


In [31]:
pd.DataFrame(cv_results["splits"])

,i_split,start,end,train,train_users,train_items,test,test_users,test_items
0,0,2021-08-16,2021-08-23,4903614,899357,15042,289252,108271,6465


In [32]:
pd.DataFrame(cv_results["metrics"])

,model,i_split,prec@1,prec@10,recall,map@10,novelty,serendipity
0,random,0,0.000194,0.000175,0.000627,0.000199,15.614943,0.000006
1,popular,0,0.071746,0.033048,0.169750,0.081908,3.706035,0.000002
2,most_raited,0,0.076244,0.030240,0.154128,0.079835,3.935765,0.000003
3,tfidf_k=5,0,0.038533,0.022477,0.104481,0.045494,9.627393,0.000267
4,tfidf_k=10,0,0.035014,0.024470,0.112454,0.043610,9.372064,0.000237
5,bm25_k=10_k1=0.05_b=0.1,0,0.000000,0.000038,0.000134,0.000028,16.330975,0.000004


In [33]:
pivot_results = (
    pd.DataFrame(cv_results["metrics"])
    .drop(columns="i_split")
    .groupby(["model"], sort=False)
    .agg(["mean", "std"])
)
mean_metric_subset = [(metric, "mean") for metric in pivot_results.columns.levels[0]]
(
    pivot_results.style
    .highlight_min(subset=mean_metric_subset, color='lightcoral', axis=0)
    .highlight_max(subset=mean_metric_subset, color='lightgreen', axis=0)
)

# Модель для сервиса (выбираем похожие на один конкретный фильм)

In [40]:
!pip install rectools

In [41]:
# Что потребуется из импортов
from datetime import datetime
from matplotlib import pyplot as plt

from pprint import pprint

import numpy as np
import pandas as pd
import pickle

from implicit.nearest_neighbours import TFIDFRecommender, BM25Recommender
from implicit.als import AlternatingLeastSquares

from rectools import Columns
from rectools.dataset import Dataset
from rectools.metrics import Precision, Recall, MeanInvUserFreq, Serendipity, calc_metrics, MAP
from rectools.models import ImplicitItemKNNWrapperModel, RandomModel, PopularModel
from rectools.model_selection import TimeRangeSplitter, cross_validate

In [42]:
!gdown 1I_uFqSR7TiGp7BfKH6wwnpmiBk1ZAzww
!gdown 1CP4deOmpaOqnwdnqZCcZBqpZjBxKDs6l
!gdown 1S0Wz20_d-HV9USfxwqmE4FRtkOovs2XK

Downloading...
From: https://drive.google.com/uc?id=1I_uFqSR7TiGp7BfKH6wwnpmiBk1ZAzww
To: /content/items.csv
100% 31.8M/31.8M [00:00<00:00, 125MB/s]
Downloading...
From: https://drive.google.com/uc?id=1CP4deOmpaOqnwdnqZCcZBqpZjBxKDs6l
To: /content/users.csv
100% 29.9M/29.9M [00:00<00:00, 180MB/s]
Downloading...
From: https://drive.google.com/uc?id=1S0Wz20_d-HV9USfxwqmE4FRtkOovs2XK
To: /content/interactions.csv
100% 180M/180M [00:02<00:00, 80.5MB/s]


In [43]:
items_df = pd.read_csv('/content/items.csv')
interactions_df = pd.read_csv('/content/interactions.csv', parse_dates=["last_watch_dt"]
    ).rename(columns={"last_watch_dt": Columns.Datetime})

In [44]:
# Готовим данные под rectools
interactions = (
    interactions_df.copy()
)

items_features = items_df[
    ['item_id', 'release_year', 'for_kids', 'age_rating', 'genres',
     'countries', 'keywords', 'title', 'description']
    ]

values = {
    'for_kids': 0,
    'age_rating': items_features.median(numeric_only=True).age_rating,
    'release_year': items_features.median(numeric_only=True).release_year,
    'countries': 'unknown',
    'keywords': 'unknown',
    'description': 'unknown',
    }

items_features = items_features.fillna(value=values)

In [45]:
interactions.head(3)

,user_id,item_id,datetime,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0


In [46]:
items_features.head(3)

,item_id,release_year,for_kids,age_rating,genres,countries,keywords,title,description
0,10711,2002.0,0.0,16.0,"драмы, зарубежные, детективы, мелодрамы",Испания,"Поговори, ней, 2002, Испания, друзья, любовь, ...",Поговори с ней,Мелодрама легендарного Педро Альмодовара «Пого...
1,2508,2014.0,0.0,16.0,"зарубежные, приключения, комедии",США,"Голые, перцы, 2014, США, друзья, свадьбы, прео...",Голые перцы,Уморительная современная комедия на популярную...
2,10716,2011.0,0.0,16.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,"Тактическая, сила, 2011, Канада, бандиты, ганг...",Тактическая сила,Профессиональный рестлер Стив Остин («Все или ...


In [47]:
# Process interactions
interactions[Columns.Weight] = np.where(interactions['watched_pct'] > 10, 3, 1)

# Split to train / test
max_date = interactions[Columns.Datetime].max()
train = interactions[interactions[Columns.Datetime] < max_date - pd.Timedelta(days=7)].copy()
test = interactions[interactions[Columns.Datetime] >= max_date - pd.Timedelta(days=7)].copy()
train.drop(train.query('total_dur < 300').index, inplace=True)
cold_users = set(test[Columns.User]) - set(train[Columns.User])
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)
test_users = test[Columns.User].unique()
catalog=train[Columns.Item].unique()

# Process item features to the form of a flatten dataframe
items = items_features.loc[items_features[Columns.Item].isin(train[Columns.Item])].copy()

items['genre'] = items['genres'].str.lower().str.replace(", ", ",", regex=False).str.split(",")
items['countries'] = items['countries'].str.lower().str.replace(", ", ",", regex=False).str.split(",")
items['keywords'] = items['keywords'].str.lower().str.replace(", ", ",", regex=False).str.split(",")

genre_feature = items[['item_id', 'genre']].explode('genre')
genre_feature.columns = ['id', 'value']
genre_feature['feature'] = 'genre'

release_year_feature = items.reindex(columns=[Columns.Item, 'release_year'])
release_year_feature.columns = ['id', 'value']
release_year_feature['feature'] = 'release_year'

for_kids_feature = items.reindex(columns=[Columns.Item, 'for_kids'])
for_kids_feature.columns = ['id', 'value']
for_kids_feature['feature'] = 'for_kids'

age_rating_feature = items.reindex(columns=[Columns.Item, 'age_rating'])
age_rating_feature.columns = ['id', 'value']
age_rating_feature['feature'] = 'age_rating'

countries_feature = items[['item_id', 'countries']].explode('countries')
countries_feature.columns = ['id', 'value']
countries_feature['feature'] = 'countries'

keywords_feature = items[['item_id', 'keywords']].explode('keywords')
keywords_feature.columns = ['id', 'value']
keywords_feature['feature'] = 'keywords'

item_features = pd.concat(
    (genre_feature, release_year_feature, for_kids_feature,
     age_rating_feature, countries_feature, keywords_feature)
)

In [48]:
train.head(3)

,user_id,item_id,datetime,total_dur,watched_pct,weight
0,176549,9506,2021-05-11,4250,72.0,3
1,699317,1659,2021-05-29,8317,100.0,3
3,864613,7638,2021-07-05,14483,100.0,3


In [49]:
item_features.head(3)

,id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre


In [50]:
# Make rectools.Dataset from pandas.DataFrame
dataset_item_features = Dataset.construct(
    interactions_df=train,
    item_features_df=item_features,
    cat_item_features=['genre', 'countries', 'keywords'],
)

In [51]:
# Make dict with metrics
metrics_name = {
    'MAP': MAP,
    'Novelty': MeanInvUserFreq,
    'Serendipity': Serendipity
}

metrics = {}
for metric_name, metric in metrics_name.items():
    for k in (1, 10):
        metrics[f'{metric_name}@{k}'] = metric(k=k)

In [52]:
metrics

{'MAP@1': MAP(k=1, divide_by_k=False),
 'MAP@10': MAP(k=10, divide_by_k=False),
 'Novelty@1': MeanInvUserFreq(k=1),
 'Novelty@10': MeanInvUserFreq(k=10),
 'Serendipity@1': Serendipity(k=1),
 'Serendipity@10': Serendipity(k=10)}

In [53]:
K_RECOS = 10

In [54]:
# Сделаем две модели (повыше MAP или Novelty)
model_tfidf = ImplicitItemKNNWrapperModel(model=TFIDFRecommender(K=10))

In [55]:
%%time
model_tfidf.fit(dataset_item_features)
recos_tfidf = model_tfidf.recommend(
    users=test_users,
    dataset=dataset_item_features,
    k=K_RECOS,
    filter_viewed=True,
)

CPU times: user 34.7 s, sys: 532 ms, total: 35.2 s
Wall time: 46.1 s


In [56]:
model_bm25 = ImplicitItemKNNWrapperModel(model=BM25Recommender(K=15, K1=1.5, B=0.9))

In [57]:
%%time
model_bm25.fit(dataset_item_features)
recos_bm25 = model_bm25.recommend(
    users=test_users,
    dataset=dataset_item_features,
    k=K_RECOS,
    filter_viewed=True,
)

CPU times: user 30.7 s, sys: 259 ms, total: 30.9 s
Wall time: 33.9 s


In [58]:
metric_values_tfidf = calc_metrics(metrics, recos_tfidf, test, train, catalog)
metric_values_tfidf['model'] = 'tfidf'

In [59]:
metric_values_bm25 = calc_metrics(metrics, recos_bm25, test, train, catalog)
metric_values_bm25['model'] = 'bm25'

In [60]:
metrics_df = (
    pd.concat(
        [pd.DataFrame(metric_values_tfidf, index=[0]),
         pd.DataFrame(metric_values_bm25, index=[1])]
        )
    .set_index('model')
)

(
    metrics_df
    .style
    .highlight_min(color='lightcoral', axis=0)
    .highlight_max(color='lightgreen', axis=0)
)

,MAP@1,MAP@10,Novelty@1,Novelty@10,Serendipity@1,Serendipity@10
model,,,,,,
tfidf,0.039328,0.074693,5.292342,6.337043,0.000436,0.000233
bm25,0.012038,0.026210,9.108759,9.079356,0.000885,0.000460


In [61]:
# # сохраним модельку
# with open('knn_bm25_model.npz', 'wb') as f:
#     model_bm25.model.save(f)

# ## попробуем загрузить
# # инит модели
# model_bm25_loaded = ImplicitItemKNNWrapperModel(model=BM25Recommender(K=15, K1=1.5, B=0.9))

# # сделаем подобие датасета для того, чтобы заставить класс ImplicitItemKNNWrapperModel
# # показывать атрибут model
# dummy_dataset = Dataset.construct(
#     interactions_df=train[train['item_id'] == 1659].head(1),
#     item_features_df=item_features[item_features['id'] == 1659],
#     cat_item_features=['genre', 'countries', 'keywords'],
# )

# # фективное обучение
# model_bm25_loaded.fit(dummy_dataset)

# # теперь он дает нам загрузить из файла
# model_bm25_loaded.model.load('knn_bm25_model.npz')

# # делаем инференс и считаем метрики
# # на этом шаге у меня три раза подряд просто крашилась сессия в ноутбуке
# recos_loaded = model_bm25_loaded.recommend(
#     users=test_users,
#     dataset=dataset_item_features,
#     k=10,
#     filter_viewed=True,
# )
# calc_metrics(metrics, recos_loaded, test, train, catalog)

In [62]:
# сохраним модельку в pickle
with open('knn_bm25_model.pkl', 'wb') as f:
    pickle.dump(model_bm25, f)

In [89]:
with open('knn_bm25_model.pkl', 'rb') as f:
    my_model = pickle.load(f)

In [64]:
recos_loaded = model_bm25.recommend(
    users=test_users,
    dataset=dataset_item_features,
    k=10,
    filter_viewed=True,
)
calc_metrics(metrics, recos_loaded, test, train, catalog)

{'MAP@1': 0.012037760367651728,
 'MAP@10': 0.026209558061077904,
 'Novelty@1': 9.108759489344077,
 'Novelty@10': 9.079356360472035,
 'Serendipity@1': 0.0008854210773294672,
 'Serendipity@10': 0.0004595315419986634}

In [65]:
recos_loaded = model_bm25_loaded_rec.recommend(
    users=test_users,
    dataset=dataset_item_features,
    k=10,
    filter_viewed=True,
)
calc_metrics(metrics, recos_loaded, test, train, catalog)

{'MAP@1': 0.012037760367651728,
 'MAP@10': 0.026209558061077904,
 'Novelty@1': 9.108759489344077,
 'Novelty@10': 9.079356360472035,
 'Serendipity@1': 0.0008854210773294672,
 'Serendipity@10': 0.0004595315419986634}

### Фильмы, для которых делаем рекомендации

In [66]:
# Поисковик по фильмам
items_features[items_features['item_id'].isin(catalog[120:130])][['item_id', 'title']]

,item_id,title
653,12589,Власть в ночном городе. Книга вторая: Призрак
1477,7347,Восточный ветер 3: Наследие Оры
4032,10551,Мститель
6689,4151,Секреты семейной жизни
6906,4209,Эмма.
7486,8636,Белый снег
8964,3819,Научи меня жить
12049,3090,Питер FM
12336,6939,Артур и Мерлин: Рыцари Камелота
12540,14264,Пираты Карибского Моря: Сундук Мертвеца


In [67]:
infer_list = [9506, 1659, 14264]

In [68]:
items_features[items_features['item_id'].isin(infer_list)].drop('description', axis=1)

,item_id,release_year,for_kids,age_rating,genres,countries,keywords,title
8760,9506,2013.0,0.0,0.0,"фэнтези, мультфильм, музыкальные",США,"королева, мюзикл, принцесса, предательство, сн...",Холодное сердце
12540,14264,2006.0,0.0,12.0,"боевики, фэнтези, приключения",США,"ведьма, гадалка, неволя, экзотический остров, ...",Пираты Карибского Моря: Сундук Мертвеца
14660,1659,2014.0,0.0,6.0,"мультфильм, фэнтези, приключения, комедии",Россия,"2014, россия, три, богатыря, ход, конем",Три богатыря. Ход конем


### Предсказания моделью TFIDF

In [69]:
# сделаем предсказания item to item моделью tfidf
item_to_item_rec_tfidf = model_tfidf.recommend_to_items(
    target_items=infer_list,
    dataset=dataset_item_features,
    k=10
    )

In [70]:
item_to_item_rec_tfidf = item_to_item_rec_tfidf.merge(items_features[['item_id', 'title']], on='item_id', how='left')

### Предсказания моделью BM25

In [71]:
# сделаем предсказания item to item моделью bm25
item_to_item_rec_bm25 = model_bm25.recommend_to_items(
    target_items=infer_list,
    dataset=dataset_item_features,
    k=10
    )

In [72]:
item_to_item_rec_bm25 = item_to_item_rec_bm25.merge(items_features[['item_id', 'title']], on='item_id', how='left')

In [134]:
item_to_item_rec_bm25

,target_item_id,item_id,score,rank,title
0,9506,7582,5121.572805,1,Холодное сердце II
1,9506,1105,3562.886750,2,Рапунцель: Запутанная история
2,9506,334,2277.038267,3,Храбрая сердцем
3,9506,5569,2256.244391,4,Русалочка
4,9506,10761,2166.706250,5,Моана
5,9506,11539,2158.550379,6,Красавица и чудовище
6,9506,13185,2080.465016,7,Хороший динозавр
7,9506,11689,2018.790387,8,Снежная Королева
8,9506,11664,1841.077900,9,Балерина
9,9506,12421,1823.264885,10,Аладдин


### Сравнение предсказаний

In [73]:
item_to_item_rec_tfidf[['item_id', 'rank', 'title']].join(
    item_to_item_rec_bm25[['item_id', 'rank', 'title']],
    lsuffix='_tfidf',
    rsuffix='_bm25',
    how='outer'
    )

,item_id_tfidf,rank_tfidf,title_tfidf,item_id_bm25,rank_bm25,title_bm25
0,7582.0,1.0,Холодное сердце II,7582,1,Холодное сердце II
1,1105.0,2.0,Рапунцель: Запутанная история,1105,2,Рапунцель: Запутанная история
2,10761.0,3.0,Моана,334,3,Храбрая сердцем
3,334.0,4.0,Храбрая сердцем,5569,4,Русалочка
4,5569.0,5.0,Русалочка,10761,5,Моана
5,13243.0,6.0,Головоломка,11539,6,Красавица и чудовище
6,16166.0,7.0,Зверополис,13185,7,Хороший динозавр
7,13185.0,8.0,Хороший динозавр,11689,8,Снежная Королева
8,3182.0,9.0,Ральф против Интернета,11664,9,Балерина
9,13586.0,1.0,Три богатыря. Принцесса Египта,12421,10,Аладдин


## Наработки для прод кода

In [121]:
class ItemToItemKnnTFIDF(ImplicitItemKNNWrapperModel):
    def __init__(self, k_nn: int=10):
        self.k_nn = k_nn
        super().__init__(model=TFIDFRecommender(K=self.k_nn))
    def save_model(self, file_path: str='my_model.pkl'):
        with open(file_path, 'wb') as f:
            pickle.dump(self, f)

In [124]:
def load_model(file_path: str='my_model.pkl'):
    with open(file_path, 'rb') as f:
        model = pickle.load(f)
    return model

In [129]:
my_model = ItemToItemKnnTFIDF(k_nn=10)
my_model.fit(dataset_item_features)
recos_my_model = my_model.recommend(
    users=test_users,
    dataset=dataset_item_features,
    k=10,
    filter_viewed=True,
)
metric_values_my_model = calc_metrics(metrics, recos_my_model, test, train, catalog)

In [130]:
metric_values_my_model

{'MAP@1': 0.03932761202533744,
 'MAP@10': 0.07469347284065676,
 'Novelty@1': 5.292342496370695,
 'Novelty@10': 6.337042688400494,
 'Serendipity@1': 0.00043585410725500794,
 'Serendipity@10': 0.00023346086511134834}

In [132]:
my_model.save_model('my_model_test.pkl')
my_model_test = load_model('my_model_test.pkl')

In [133]:
recos_my_model = my_model_test.recommend(
    users=test_users,
    dataset=dataset_item_features,
    k=10,
    filter_viewed=True,
)
metric_values_my_model = calc_metrics(metrics, recos_my_model, test, train, catalog)

In [76]:
class ItemToItemKnnBM25(ImplicitItemKNNWrapperModel):
    def __init__(self, k_nn: int=15, k1: float=1.5, b: float=0.9):
        self.k_nn = k_nn
        self.k1 = k1
        self.b = b
        super().__init__(model=BM25Recommender(K=self.k_nn, K1=self.k1, B=self.b))
    def save_model(self, file_path: str='my_model.pkl'):
        with open(file_path, 'wb') as f:
            pickle.dump(self, f)
    def load_model(self, file_path: str='my_model.pkl')
        with open(file_path, 'rb') as f:
            self = pickle.load(f)

In [77]:
my_model = ItemToItemKnnBM25()
my_model.fit(dataset_item_features)
recos_my_model = my_model.recommend(
    users=test_users,
    dataset=dataset_item_features,
    k=10,
    filter_viewed=True,
)
metric_values_my_model = calc_metrics(metrics, recos_my_model, test, train, catalog)